# Spaceshipe Titanic v.2

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
train=pd.read_csv('train.csv')

In [3]:
df=train.copy()

In [4]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

## Homplanet

In [5]:
# group_home_dict
with open('group_home_dict.pkl','rb') as f:
    group_home_dict=pickle.load(f)
# group2_home_dict
with open('group2_home_dict.pkl','rb') as f:
    group2_home_dict=pickle.load(f)   
# dest_home_dict
with open('dest_home_dict.pkl','rb') as f:
    dest_home_dict=pickle.load(f)
# home_deck_dict
with open('home_deck_dict.pkl','rb') as f:
    home_deck_dict=pickle.load(f)
#group_side_dict
with open('group_side_dict.pkl','rb') as f:
    group_side_dict=pickle.load(f)
#dest_dict
with open('dest_dict.pkl','rb') as f:
    dest_dict=pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'group_home_dict.pkl'

In [ ]:
with open('deckle.pkl','rb') as f:
    deckle=pickle.load(f)
with open('Destinationle.pkl','rb') as f:
    Destinationle=pickle.load(f)
with open('HomePlanetle.pkl','rb') as f:
    HomePlanetle=pickle.load(f)
with open('sidele.pkl','rb') as f:
    sidele=pickle.load(f)

In [ ]:
deckle.classes_

In [6]:
def preprocess(data):
    
    data['Group']=data['PassengerId'].str[:4]


    data['HomePlanet'].fillna(data['Group'].map(group_home_dict),inplace=True)
    data['Group2']=data['Name'].str.split(' ').str[-1]

    data['HomePlanet'].fillna(data['Group2'].map(group2_home_dict),inplace=True)

    data['HomePlanet']=data['HomePlanet'].fillna(data['Destination'].map(dest_home_dict))

    data['VIP'].fillna(False,inplace=True)
    fee_list=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
    for col in fee_list:
        data[col].fillna(0,inplace=True)
    data['fee']=data['RoomService']+data['FoodCourt']+data['ShoppingMall']+data['Spa']+data['VRDeck']
    data['fee'].fillna(0,inplace=True)

    data['deck']=data['Cabin'].str.split('/').str[0]

    data['deck'].fillna(data['HomePlanet'].map(home_deck_dict),inplace=True)

    data['side']=data['Cabin'].str.split('/').str[-1]


    data['side'].fillna(data['Group'].map(group_side_dict),inplace=True)
    data['side'].fillna('S',inplace=True)


    data['Destination'].fillna(data['Group'].map(dest_dict),inplace=True)
    data['Destination'].fillna('TRAPPIST-1e',inplace=True)
    data['CryoSleep']=np.where(data['fee']>0,False,True)


    age_med=27
    data['Age'].fillna(age_med,inplace=True)


    del_col=['PassengerId','Cabin','Group','Name','Group2']
    data.drop(del_col,axis=1,inplace=True)

    data['HomePlanet']=HomePlanetle.transform(data['HomePlanet'])
    data['deck']=deckle.transform(data['deck'])
    data['side']=sidele.transform(data['side'])
    data['Destination']=Destinationle.transform(data['Destination'])
    
    
    
    
    
    
    
    return data


In [7]:
df=preprocess(df)

NameError: name 'group_home_dict' is not defined

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer

In [9]:
preproc_t=FunctionTransformer(preprocess)

In [10]:
pipe=make_pipeline(preproc_t,XGBClassifier())

In [15]:
model=XGBClassifier()

In [16]:
model.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)>

In [11]:
param_grid={
    'xgbclassifier__max_depth':[3,6,9,10],
    'xgbclassifier__learning_rate':[0.001,0.01,0.1,0.2,0.3,1]
    'xgbclassifier__gamma':[1,2,3]
}

In [12]:
gridcv=GridSearchCV(pipe,param_grid=param_grid,n_jobs=-1)

In [13]:
X=pd.read_csv('train.csv').drop('Transported',axis=1)
y=pd.read_csv('train.csv')['Transported']

In [14]:
gridcv.fit(X,y)

C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
120 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\pipeline.py", line 348, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\user\anaconda3

NameError: name 'group_home_dict' is not defined

In [35]:
gridcv.best_score_

0.8009914696421564

In [36]:
X_test=pd.read_csv('test.csv')

In [37]:
y_pred=gridcv.best_estimator_.predict(X_test)

In [38]:
y_pred.astype(bool)

array([ True, False,  True, ...,  True,  True,  True])

In [40]:
jiye=pd.read_csv('sample_submission.csv')

In [41]:
jiye['Transported']=y_pred.astype(bool)

In [42]:
jiye.to_csv('jiye2.csv',index=False)